In [25]:
import sys
sys.path.insert(0, '/home/oscarwu/code/chemprop_developing')

import pickle as pkl
import numpy as np
import pandas as pd

In [26]:
# load an example
df_smi = pd.read_csv('esol.csv')
df_smi.iloc[[7]]

,smiles,logSolubility
7,CC#N,0.26


In [29]:
smiles = df_smi.iloc[[7]]['smiles'].values.tolist()

In [30]:
smiles

['CC#N']

In [45]:
from chemprop.features.utils import load_features, load_valid_atom_or_bond_features

In [52]:
bond_features = load_valid_atom_or_bond_features('esol_wb97xd_bond_RBF_features.pkl', smiles)[7]

In [53]:
bond_features

array([[1.78286192e-06, 9.98036728e-06, 4.95518646e-05, 2.18201706e-04,
        8.52198850e-04, 2.95194600e-03, 9.06902365e-03, 2.47113984e-02,
        5.97198590e-02, 1.28004397e-01, 2.43341209e-01, 4.10290158e-01,
        6.13551750e-01, 8.13759292e-01, 9.57250021e-01, 9.98710166e-01,
        9.24140958e-01, 7.58440698e-01, 5.52064348e-01, 3.56403891e-01,
        2.04070314e-01, 1.03633912e-01, 4.66776103e-02, 1.86466147e-02,
        6.60657003e-03, 2.07604494e-03, 5.78604917e-04, 1.43025065e-04,
        3.13564470e-05, 6.09714050e-06, 1.05150229e-06, 1.60834385e-07,
        2.18188695e-08, 2.62524748e-09, 2.80151478e-10, 2.65155260e-11,
        2.22583127e-12, 1.65717698e-13, 1.09428440e-14, 6.40879215e-16,
        3.32894659e-17, 1.53363542e-18, 6.26645744e-20, 2.27094555e-21,
        7.29921255e-23, 2.08079819e-24, 5.26100331e-26, 1.17975506e-27,
        2.34638778e-29, 4.13897104e-31, 6.80038410e-05, 1.92992280e-04,
        5.15808818e-04, 1.29831453e-03, 3.07760874e-03, 6.870513

In [57]:
bond_features.shape

(5, 200)

In [59]:
mask_features_extra_batch([bond_features], [[1, 0, 1, 1]])

[array([[1.78286192e-06, 9.98036728e-06, 4.95518646e-05, 2.18201706e-04,
         8.52198850e-04, 2.95194600e-03, 9.06902365e-03, 2.47113984e-02,
         5.97198590e-02, 1.28004397e-01, 2.43341209e-01, 4.10290158e-01,
         6.13551750e-01, 8.13759292e-01, 9.57250021e-01, 9.98710166e-01,
         9.24140958e-01, 7.58440698e-01, 5.52064348e-01, 3.56403891e-01,
         2.04070314e-01, 1.03633912e-01, 4.66776103e-02, 1.86466147e-02,
         6.60657003e-03, 2.07604494e-03, 5.78604917e-04, 1.43025065e-04,
         3.13564470e-05, 6.09714050e-06, 1.05150229e-06, 1.60834385e-07,
         2.18188695e-08, 2.62524748e-09, 2.80151478e-10, 2.65155260e-11,
         2.22583127e-12, 1.65717698e-13, 1.09428440e-14, 6.40879215e-16,
         3.32894659e-17, 1.53363542e-18, 6.26645744e-20, 2.27094555e-21,
         7.29921255e-23, 2.08079819e-24, 5.26100331e-26, 1.17975506e-27,
         2.34638778e-29, 4.13897104e-31, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000

In [55]:
from typing import List, Optional, Union, Tuple

def mask_features_extra_batch(features_extra_batch: List[np.ndarray], keep_features_batch: List[List[bool]], feature_length: int = 50) -> List[np.ndarray]:
    """
    Masks certain features in each np.ndarray in features_extra_batch based on the corresponding boolean vector in keep_features_batch.

    :param features_extra_batch: A list of 2D numpy arrays, each with shape (num_atoms, feature_length) where feature_length is a multiple of 50.
    :param keep_features_batch: A list of lists of boolean vectors, each indicating which features to keep for the corresponding np.ndarray in features_extra_batch.
    :return: A list of masked 2D numpy arrays with the same shapes as the corresponding entries in features_extra_batch.
    """
    return [mask_features_extra(features_extra, keep_features, feature_length) for features_extra, keep_features in zip(features_extra_batch, keep_features_batch)]

def mask_features_extra(features_extra: np.ndarray, keep_features: List[bool], feature_length: int = 50) -> np.ndarray:
    """
    Masks certain features in features_extra(atom/bond) based on the keep_features boolean vector.

    :param features_extra: A 2D numpy array with shape (num_atoms or num_bonds, feature_vector) where feature_vector is a multiple of feature_length.
    :param keep_features: A boolean vector indicating which features to keep.
    :return: A masked 2D numpy array with the same shape as features_extra.
    """
    _, feature_vector = features_extra.shape
    num_features = feature_vector // feature_length  # Each feature has length 50

    if len(keep_features) != num_features:
        raise ValueError("Length of keep_features does not match the number of features in features_extra.")

    # Create a mask to apply to features_extra
    mask = np.ones_like(features_extra)

    for i, keep in enumerate(keep_features):
        if not keep:
            mask[:, i*50:(i+1)*50] = 0

    # Apply the mask to features_extra
    masked_features = features_extra * mask

    return masked_features

In [23]:
# Example SHAP Analysis
# Imports
from chemprop.args import TrainArgs
from chemprop.models.model import MoleculeModel

import torch
from chemprop.utils import load_checkpoint, load_scalers
from chemprop.features import set_extra_bond_fdim

# Load TrainArgs and MoleculeModel
path = "/home/oscarwu/code/chemprop_developing/developing/shap_v1/data/model.pt"
state = torch.load(path, map_location=lambda storage, loc: storage)
train_args = TrainArgs()
train_args.from_dict(vars(state["args"]), skip_unsettable=True)
set_extra_bond_fdim(train_args.bond_features_size)
model = load_checkpoint(path)
scalers = load_scalers(path)

# Create a MoleculeModel Wrapper
class MoleculeModelWrapper:
    def __init__(self, model, train_args, scalers, smiles, shap, features_batch, 
                 atom_descriptors_batch, atom_features_batch, bond_descriptors_batch, bond_features_batch):
        self.model = model
        self.train_args = train_args
        self.target_scaler, self.mol_feature_scaler, _, _, _ = scalers
        self.smiles = smiles
        self.shap = shap
        self.features_batch = self.mol_feature_scaler.transform(features_batch) if features_batch is not None else None
        self.atom_descriptors_batch = atom_descriptors_batch
        self.atom_features_batch = atom_features_batch
        self.bond_descriptors_batch = bond_descriptors_batch
        self.bond_features_batch = bond_features_batch
        

    def __call__(self, feature_choice):
        batch = [[smi] for smi in self.smiles]
        extra_keep_features_batch = [feature_choice[0:20]] # 20 extra molecular features
        extra_atom_keep_descriptors_batch = [feature_choice[20:33]] # 13 atom descriptors
        extra_bond_keep_descriptors_batch = None # 0 bond descriptors
        extra_atom_keep_features_batch = None # 0 atom features
        extra_bond_keep_features_batch = [feature_choice[33:37]] # 4 bond features
        chemprop_atom_keep_features = [feature_choice[37:45]] # 8 chemprop atom features
        chemprop_bond_keep_features = [feature_choice[45:49]] # 4 chemprop bond features
        
        output = self.model(batch, 
                            self.features_batch, 
                            self.atom_descriptors_batch, 
                            self.atom_features_batch, 
                            self.bond_descriptors_batch, 
                            self.bond_features_batch, 
                            shap, 
                            extra_keep_features_batch, 
                            extra_atom_keep_descriptors_batch, 
                            extra_bond_keep_descriptors_batch, 
                            extra_atom_keep_features_batch, 
                            extra_bond_keep_features_batch, 
                            chemprop_atom_keep_features, 
                            chemprop_bond_keep_features)
        
        return self.target_scaler.inverse_transform(output).flatten()
        

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "encoder.encoder.0.atom_descriptors_layer.weight".
Loading pretrained parameter "encoder.encoder.0.atom_descriptors_layer.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Loading pretrained parameter "readout.7.weight".
Loading pretrained parameter "readout.7.bias".
